In [1]:
%load_ext cython

In [2]:
import numpy as np

In [3]:
%%cython
# cython: infer_types = True
# cython: boundscheck = False
# cython: wraparound = False
cimport cython
from numpy import zeros, double as npy_double
from libc.math cimport sqrt

cdef void mean_sd_1d(const double[:] x, double* mean, double* std):
    cdef Py_ssize_t n = x.size, i

    cdef double k = x[0]
    cdef double Ex = 0., Ex2 = 0.
    mean[0] = 0.

    for i in range(n):
        mean[0] += x[i]
        Ex += x[i] - k
        Ex2 += (x[i] - k)**2
    
    std[0] = sqrt((Ex2 - (Ex**2 / n)) / (n - 1))
    mean[0] /= n
    
    
cpdef CID(const double[:, :, :] x, bint normalize):
    cdef Py_ssize_t m = x.shape[0], n = x.shape[1], p = x.shape[2], i, j, k

    dist = zeros((m, p), dtype=npy_double)
    cdef double[:, ::1] cid = dist
    cdef double mu = 0., sigma = 0.

    for i in range(m):
        for k in range(p):
            if normalize:
                mu = 0.
                sigma = 0.
                mean_sd_1d(x[i, :, k], &mu, &sigma)
                if sigma != 0.:
                    for j in range(1, n):
                      cid[i, k] += ((x[i, j, k] - x[i, j-1, k]) / sigma)**2
            else:  # if not normalizing
                for j in range(1, n):
                    cid[i, k] += (x[i, j, k] - x[i, j-1, k])**2
    
    for i in range(m):
        for k in range(p):
            cid[i, k] = sqrt(cid[i, k])
    
    return dist

In [6]:
from complexityinvariantdistance import complexityinvariantdistance as fcid

In [7]:
x = np.random.rand(50000, 150, 3)
xf = np.asfortranarray(x.transpose([1, 2, 0]))

In [8]:
%timeit CID(x, True)

83.4 ms ± 11.6 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%timeit fcid(xf, True)

37.6 ms ± 86.2 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
np.allclose(fcid(xf, True).T, CID(x, True))

True